In [0]:
import requests
import uuid
from datetime import datetime
from pyspark.sql.functions import col, udf, explode
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType, ArrayType
from pyspark.sql.functions import from_unixtime
from delta.tables import *

In [0]:
df = spark.sql("select * from cities limit 5") #selecting from cities table 

In [0]:
df = df.withColumn('id', col('id').cast('int')) #changing the id column to int 

In [0]:
%run ../My_etl_module/logger 


In [0]:
!pip install python-dotenv

In [0]:
%run "/sagar.rimal@acme.edu.np/Weather_ETL/Api_module"

In [0]:
#load bronze table(raw data table) function call from api module
df_bronze = get_weather('RAW', 'weather_bronze', df) #calling get_weather function to load api data into raw table

In [0]:
display(df_bronze)

In [0]:
%sql
select * from weather_log_table_new

In [0]:
%sql
select * from weather_bronze

In [0]:
@keep_log #falttening and loading into silver table
def get_process(df):
    df = df.select(
            col('result.dt'),
            from_unixtime(col('result.dt')).alias('date'),
            col('result.id').alias('city_id'),
            col('result.name').alias('city_name'),
            col('result.timezone'),
            col('result.sys.country'),
            col('result.coord.lat'),
            col('result.coord.lon'),
            col('result.main.temp'),
            col('result.main.temp_min'),
            col('result.main.temp_max'),
            col('result.main.pressure'),
            col('result.main.humidity'),
            col('result.visibility'),
            col('result.wind.speed').alias("wind_speed"),
            col('result.wind.deg').alias("wind_deg"),
            col('result.wind.gust').alias("wind_gust"),
            col('result.clouds.all').alias("clouds_all"))
    
    start = datetime.fromtimestamp(df.selectExpr("min(dt)").first()[0])
    end = datetime.fromtimestamp(df.selectExpr("max(dt)").first()[0])
    
    return df, start, end
    

In [0]:
df_silver = get_process('PROCESSED', 'weather_silver', df_bronze)

In [0]:
display(df_silver)

In [0]:
%sql
select * from weather_log_table_new

In [0]:
%sql
select * from weather_silver

In [0]:
%run "/sagar.rimal@acme.edu.np/Weather_ETL/dim_loading"

In [0]:
# Call the function to create the date dimension table
create_date_dimension("2023-01-01", "2024-12-31")

In [0]:
%sql
select * from dim.dim_date

In [0]:
# Call the function to create the time dimension table
create_time_dimension("00:00:00", "23:59:59", "1 hour")

In [0]:
%sql
select * from dim.dim_time;

In [0]:
# Call the function to insert data into the dim_city table
insert_dim_city_table()


In [0]:
spark.sql("select * from dim.dim_city").show()

In [0]:
%run "/sagar.rimal@acme.edu.np/Weather_ETL/fact-loading"


In [0]:
# Call the function to insert the latest hourly weather data and update the log as usual
df_fact_hourly=insert_latest_hourly_weather_data('FACT','fact.hourly_weather_data')


In [0]:
%sql
select * from fact.hourly_weather_data;

In [0]:
# Call the function to insert the daily weather data and update the log as well
df_fact = insert_latest_daily_weather_data('FACT', 'fact.daily_weather_data')

In [0]:
%sql
select * from fact.daily_weather_data;